In [1]:
import os, sys
sys.path.append('/data/spark15/python/')

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 9)
import seaborn as sb
import numpy as np
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import *
import pyspark.sql.functions

sc = SparkContext("local", "hospital_compare")
sqlContext = SQLContext(sc)
hc = HiveContext(sc)

In [3]:
print(hc.tableNames())

[u'effective_care', u'effective_care_scores', u'hospitals', u'hospitals_reduced', u'measures', u'readmissions', u'readmissions_reduced', u'survey_responses', u'survey_responses_reduced']


### Which procedures have the greatest variability between hospitals?

In [4]:
hospitals = hc.sql('select * from hospitals_reduced').toPandas()
effective_care = hc.sql('select * from effective_care_scores').toPandas()

In [5]:
df = pd.merge(hospitals, effective_care, how='inner', left_on='provider_id', right_on='provider_id')

In [15]:
results = df.set_index(['provider_id', 'hospital_name', 'measure_id'])[['score']].unstack()
results = results.apply(pd.to_numeric)
results.loc[:, 'Average_Score'] = results.mean(axis=1)
results.loc[:, 'StDev_Score'] = results.std(axis=1)

In [18]:
results.std(axis=0).sort_values(ascending=False)

               measure_id            
score          VTE_5                     4.949747
               OP_23                     3.817254
               IMM_3_OP_27_FAC_ADHPCT    3.492850
               OP_29                     3.289455
               IMM_2                     3.010084
               OP_30                     2.991534
               ED_2b                     2.833043
               VTE_6                     2.383159
               OP_5                      2.266249
               STK_4                     2.242448
               OP_20                     2.176601
               PC_01                     2.028334
Average_Score                            1.731197
score          OP_22                     1.555471
StDev_Score                              1.265789
score          OP_31                     0.000000
               ED_1b                          NaN
               OP_1                           NaN
               OP_18b                         NaN
            

In [22]:
zip(df['measure_id'].unique(), df['measure_name'].unique())

[(u'ED_1b', u'ED1'),
 (u'ED_2b', u'ED2'),
 (u'IMM_2', u'Immunization for influenza'),
 (u'IMM_3_OP_27_FAC_ADHPCT',
  u'Healthcare workers given influenza vaccination'),
 (u'OP_18b', u'OP 18'),
 (u'OP_20', u'Door to diagnostic eval'),
 (u'OP_21', u'Median time to pain med'),
 (u'OP_22', u'Left before being seen'),
 (u'OP_29',
  u'Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients'),
 (u'OP_30',
  u'Endoscopy/polyp surveillance: colonoscopy interval for patients with a history of adenomatous polyps - avoidance of inappropriate use'),
 (u'PC_01',
  u'Percent of newborns whose deliveries were scheduled early (1-3 weeks early), when a scheduled delivery was not medically necessary'),
 (u'STK_4', u'Thrombolytic Therapy'),
 (u'VTE_5', u'Warfarin therapy discharge instructions'),
 (u'VTE_6',
  u'Hospital acquired potentially preventable venous thromboembolism'),
 (u'OP_4', u'Aspirin at Arrival'),
 (u'OP_5', u'Median Time to ECG'),
 (u'O